In [ ]:
import pyfits,glob,time
from subprocess import call,check_call
#from ds9 import *
from numpy import *
import numpy as np
from scipy.ndimage import zoom,shift
import scipy.ndimage as snd
from matplotlib.pyplot import *
import seaborn as sb
from itertools import *
from matplotlib.colors import LogNorm
%matplotlib inline

In [ ]:
def mosaic_combine_ITL(filename):
    "Takes in a multi-extension mosaic fits file and the dimensions of the mosaic and returns an array with all chips combined"
    dim_x,dim_y=8,2
    hdr0 = pyfits.getheader(filename,0)
    numccds = hdr0['nextend']
    sizex,sizey = hdr0['IMG_COLS'],hdr0['IMG_ROWS']
    overscanx,overscany = hdr0['OVR_COLS'],hdr0['OVR_ROWS']
    prescanx,prescany = hdr0['PRE_COLS'],hdr0['PRE_ROWS']
    sizex,sizey = sizex,sizey
    
    ccdarray = np.zeros((sizey*dim_y,sizex*dim_x),dtype='float')
        
    for ccdnum in range(numccds):
        ccdseg = pyfits.getdata(filename,ccdnum+1)
        med_overscan=median(ccdseg[:,-overscanx:])
        ccdseg=ccdseg[prescany:-overscany,prescanx:-overscanx]-med_overscan
        j=ccdnum/dim_x   # depends on int division
        if ccdnum<dim_x:
            i=dim_x-ccdnum-1
            ccdseg=ccdseg[:,::-1]
        else:
            i=ccdnum-dim_x
            ccdseg=ccdseg[::-1,:]
        ccdarray[j*sizey:(j+1)*sizey,i*sizex:(i+1)*sizex]=ccdseg
    ccdbottom=ccdarray[0:sizey*dim_y/2,0:sizex*dim_x]
    ccdarray[0:sizey*dim_y/2,0:sizex*dim_x]=ccdbottom[:,::-1]
    return ccdarray

def make_ell_reg_from_xy(filename):
    """Takes in X & Y coordinates and a text tag and outputs a DS9 region file"""
    foocat=pyfits.getdata(filename+'.cat',"LDAC_OBJECTS")
    xreg,yreg=foocat['XWIN_IMAGE'],foocat['YWIN_IMAGE']
    radius1=7*np.ones(len(xreg))
    radius2=foocat['B_IMAGE']/foocat['A_IMAGE']*radius1
    anglereg=foocat['THETA_IMAGE']
    text_tag=['']*len(xreg)
    reg_filename=filename+'.reg'
    f = open(reg_filename,'w')
    f.write('image'+' \n')
    for i in range(len(xreg)):
        thetext=text_tag[i]
        f.write('ellipse '+str(xreg[i])+' '+str(yreg[i])+' '+str(radius1[i])+' '+str(radius2[i])+
                ' '+str(anglereg[i])+' #text="'+str(thetext)+'"\n')
    f.close()

    
def run_sextractor_on_mosaic(fitsfilename):
    #from mydefs import make_ell_reg_from_xy
    #from mydefs import make_reg_from_ldac
    import pyfits,numpy
    from subprocess import call
    configfile='/Users/cslage/.ipython/notebooks/spots/30-micron-pinhole.sex'
    paramfile='/Users/cslage/.ipython/notebooks/spots/default-array_dither.param'
    #hdr0=pyfits.open(fitsfilename)[0].header
    #pdiode=hdr0['MONDIODE']
    #exptime=hdr0['EXPTIME']
    #fluxval=float(pdiode)*float(exptime)
    minadu=100#5e10*fluxval
    outname=fitsfilename
    callstring=["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat',"-CATALOG_TYPE","FITS_LDAC",
               "-CHECKIMAGE_NAME",outname[:-4]+"back.fits"+','+outname[:-4]+"rms.fits",
               "-CHECKIMAGE_TYPE","BACKGROUND,BACKGROUND_RMS",
               "-DETECT_THRESH","10","-DETECT_MINAREA","10","-THRESH_TYPE","RELATIVE","-DETECT_MAXAREA","400",
               "-ANALYSIS_THRESH","2"]
    test=call(callstring)
    make_ell_reg_from_xy(fitsfilename)
    #make_ell_reg_from_xy(outname)
    #call(["rm",outname])
    #print callstring
    

def make_stamp_stack_subpix(cat,imagename,obj_inds,halfstampsize,nzoom):
    maxs = []
    sums = []
    sz=halfstampsize
    nobjs=len(obj_inds)
    img=pyfits.getdata(imagename)
    xc,yc=cat['XWIN_IMAGE'],cat['YWIN_IMAGE']
    nstamps=0
    stampstack=np.zeros((sz*nzoom*2,sz*nzoom*2))
    xsize,ysize=3900,3900
    figure(figsize=(10,5))
    for k in range(nobjs):
        ind=obj_inds[k]
        x,y=xc[ind]+.5,yc[ind]+.5
        if ((x<sz) | (x >xsize-sz-1) | (y<sz) | (y>ysize-sz-1)): 
            continue
        xf,yf=np.floor(x),np.floor(y)
        xshift,yshift=round((x-xf)*(nzoom)),round((y-yf)*(nzoom))
        onestamp=img[int(y-sz):int(y+sz),int(x-sz):int(x+sz)]
        maxs.append(onestamp.max())
        sums.append(onestamp.sum())
        onestamp=onestamp/np.max(onestamp)
        regridstamp=np.zeros((sz*nzoom*2,sz*nzoom*2))
        for i in range(sz*2):
            for j in range(sz*2):
                i0,i1=nzoom*i,nzoom*(i+1)
                j0,j1=nzoom*j,nzoom*(j+1)
                regridstamp[i0:i1,j0:j1]=onestamp[i,j]
        shiftstamp=shift(regridstamp,(-yshift+nzoom,-xshift+nzoom),order=0)
        """
        if k == 12:
            print "Ind = ",ind
            print "xc[ind], yc[ind] = ",xc[ind], yc[ind]
            print "x,y=",x,y
            print "xf,yf=",xf,yf
            print "xshift,yshift=",xshift,yshift
            #print onestamp[4,:]
            #print regridstamp[40,:]
            #print shiftstamp[42,:]
            subplot(221)
            title("onestamp (%d,%d)"%(onestamp.shape[0],onestamp.shape[1]))
            imshow(onestamp,cmap='rainbow',aspect='equal',origin='lower',interpolation='nearest'),colorbar()
            subplot(222)
            title("regridstamp (%d,%d)"%(regridstamp.shape[0],regridstamp.shape[1]))
            imshow(regridstamp,cmap='rainbow',aspect='equal',origin='lower',interpolation='nearest'),colorbar()
            subplot(223)
            title("shiftstamp (%d,%d)"%(shiftstamp.shape[0],shiftstamp.shape[1]))
            imshow(shiftstamp,cmap='rainbow',aspect='equal',origin='lower',interpolation='nearest'),colorbar()
        """
        stampstack+=shiftstamp
        nstamps+=1
    maxs = np.array(maxs)
    print "Max mean = %.2f, Max std = %.2f, N = %d"%(maxs.mean(), maxs.std(), len(maxs))
    sums = np.array(sums)
    print "Sum mean = %.2f, Sum std = %.2f, N = %d"%(sums.mean(), sums.std(), len(sums))
    stampstack=stampstack/nstamps
    return stampstack

def make_mosaic_stamp_subpix(stackfitsname,halfwin,ndiv,nzoom):
    """Takes in a fits file and searches for its catalog, then stacks object images by shifting 
    and medianing. Halfwin is the half size of the window around each object, and ndiv is how many 
    divisions to split the image into"""
    sexcat=pyfits.getdata(stackfitsname+'.cat',"LDAC_OBJECTS")
    fw=(halfwin*2*nzoom)
    xmossize,ymossize=4071,3999#4072,4000
    xgridsz,ygridsz=xmossize/ndiv*1.,ymossize/ndiv*1.
    xgridmins,ygridmins=linspace(0,xmossize,ndiv),linspace(0,ymossize,ndiv)
    allstampstacks=np.zeros(((halfwin*2*nzoom+1)*ndiv,(halfwin*2*nzoom+1)*ndiv))
    for i in range(ndiv):
        xgridmin,xgridmax=i*xgridsz,(i+1)*xgridsz
        for j in range(ndiv):
            ygridmin,ygridmax=j*ygridsz,(j+1)*ygridsz
            g=where((sexcat['XWIN_IMAGE']>xgridmin) & (sexcat['XWIN_IMAGE'] < xgridmax) &
                    (sexcat['YWIN_IMAGE']>ygridmin) & (sexcat['YWIN_IMAGE'] < ygridmax) &
                    (sexcat['FLAGS']==0))[0]
            onestamp=make_stamp_stack_subpix(sexcat,stackfitsname,g,halfwin,nzoom)
            allstampstacks[j*fw:(j+1)*fw,i*fw:(i+1)*fw]=onestamp
    return allstampstacks

def radial_profile(data, center):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    rx=linspace(0,halfstampsize*np.sqrt(2),len(nr))
    return rx,radialprofile 

def moments(data, center):
    y, x = np.indices((data.shape))
    x = (x - center[0] + 0.5) / nzoom
    y = (y - center[1] + 0.5) / nzoom
    mx = (data * x).sum() / data.sum()
    my = (data * y).sum() / data.sum()
    mx2 = (data * (x-mx) * (x-mx)).sum() / data.sum()
    my2 = (data * (y-my) * (y-my)).sum() / data.sum()
    mxy = (data * (x-mx) * (y-my)).sum() / data.sum()
    return [mx, my, mx2, my2, mxy]



In [ ]:
thedir='/Users/cslage/Research/LSST/code/galsim-newgit/GalSim/devel/brighter-fatter/drizzled_output/dr_1/'
mosfilelist=sort(glob.glob(thedir+'test_2.fits'))
print mosfilelist


In [ ]:
tstart=time.time()
for i in range(len(mosfilelist)): run_sextractor_on_mosaic(mosfilelist[i])

print time.time()-tstart

In [ ]:
#figure(figsize=(10,5))
for filename in mosfilelist:
    print filename
    nzoom=100
    halfstampsize=5
    xmin,xmax=0,200
    ymin,ymax=0,200

    cat=pyfits.getdata(filename+'.cat',"LDAC_OBJECTS")

    obj_inds=where((cat['XWIN_IMAGE']<xmax) & (cat['YWIN_IMAGE']<ymax) &
           (cat['XWIN_IMAGE']>xmin) & (cat['YWIN_IMAGE']>ymin) & (cat['FLAGS']==0))[0]
    print "Nobjs = %d"%len(obj_inds)

    stampstack=make_stamp_stack_subpix(cat,filename,obj_inds,halfstampsize,nzoom)

    subplot(121)
    imshow(stampstack,cmap='rainbow',aspect='equal',origin='lower'),colorbar()
    for i in range(1,2*halfstampsize):
        axvline(i*nzoom,color='w')
        axhline(i*nzoom,color='w')

    axis('off')
    [mx, my, mx2, my2, mxy] = moments(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))
    print "Sigmax = %.4f, Sigmay = %.4f"%(sqrt(mx2), sqrt(my2))

    subplot(122)
    rx,rp=radial_profile(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))
    plot(rx,rp, label = 'Data stacked')
    foox=np.linspace(.001,4,100)
    foosig=sqrt(mx2)
    fooy=np.exp(-foox**2/(2.0 * foosig**2))
    plot(foox,fooy,label='Gaussian')
legend(loc='upper right')
xlim(0,4.0)



In [ ]:
figure(figsize=(10,5))
for filename in mosfilelist:
    print filename
    nzoom=10
    halfstampsize=5
    xmin,xmax=0,2000
    ymin,ymax=0,2000

    cat=pyfits.getdata(filename+'.cat',"LDAC_OBJECTS")

    obj_inds=where((cat['XWIN_IMAGE']<xmax) & (cat['YWIN_IMAGE']<ymax) &
           (cat['XWIN_IMAGE']>xmin) & (cat['YWIN_IMAGE']>ymin) & (cat['FLAGS']==0))[0]
    print "Nobjs = %d"%len(obj_inds)

    stampstack=make_stamp_stack_subpix(cat,filename,obj_inds,halfstampsize,nzoom)


    subplot(121)
    imshow(stampstack,cmap='rainbow',aspect='equal',origin='lower', interpolation='nearest'),colorbar()
    for i in range(1,2*halfstampsize):
        axvline(i*nzoom,color='w')
        axhline(i*nzoom,color='w')

    axis('off')
    [mx, my, mx2, my2, mxy] = moments(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))
    print "Sigmax = %.4f, Sigmay = %.4f"%(sqrt(mx2), sqrt(my2))

    subplot(122)
    rx,rp=radial_profile(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))
    plot(rx,rp, label = 'Data stacked')
    foox=np.linspace(.001,4,100)
    foosig=sqrt(mx2)
    fooy=np.exp(-foox**2/(2.0 * foosig**2))#*27038
    plot(foox,fooy,label='Gaussian')
legend(loc='upper right')
xlim(0,4.0)

